In [7]:
import pandas as pd
import pickle

In [26]:
!pip install simplejson

     |████████████████████████████████| 81kB 843kB/s eta 0:00:011


In [14]:
# df = pd.read_pickle('fipi.pickle')

with open('cds_team_phdr_dataset.pkl', "rb") as f:
    temp = pickle.load(f)

In [18]:
print(type(temp))
print(len(temp))
print(len(temp[0]))
print(len(temp[1]))

<class 'tuple'>
2
10009
10009


In [16]:
print(temp[1])

[1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 

In [39]:
import json

task_id_offset = 17

global_counter = 0

for i in range(len(temp[0])):
    sent = temp[0][i]
    sol = temp[1][i]
    if sol == 0:
        continue
        
    task_id = task_id_offset + (global_counter % 4)

    global_counter = global_counter + 1

    subsents = sent.split('PHDR')
    out = []
    counter = 1
    choices = []
    for subsent in subsents:
        words = subsent.split(' ')
        idx = len(words) // 2
        placeholder = '({})'.format(counter)
        words.insert(idx, placeholder)
        choices.append({"id": str(counter), "placeholder": placeholder})
        counter = counter + 1
        out.append(" ".join(words))

    placeholder = '({})'.format(counter)
    choices.append({"id": str(counter), "placeholder": placeholder})
    correct = [str(counter)]
    masked_sent = placeholder.join(out)

    item = {
        "id": str(task_id),
        "score": 1,
        "text": masked_sent,
        "attachments": [],
        "question": {
            "type": "multiple_choice",
            "choices": choices,
            "min_choices": 1
        },
        "solution": {
            "correct": correct
        }
    }

    data_fn = "task_{}_custom_{}.json".format(task_id, global_counter)
    datafile = open(data_fn, "w", encoding='utf8')
    datafile.write(json.dumps(item, indent=4, ensure_ascii=False))
    datafile.close()